In [146]:
import pandas as pd
from glob import glob


In [147]:
data = pd.read_csv("../data/raw/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
data.head(10)

,epoch (ms),time (01:00),elapsed (s),x-axis (g),y-axis (g),z-axis (g)
0,1547219408431,2019-01-11T16:10:08.431,0.00,0.010,0.964,-0.087
1,1547219408511,2019-01-11T16:10:08.511,0.08,0.000,0.961,-0.069
2,1547219408591,2019-01-11T16:10:08.591,0.16,0.001,0.974,-0.087
3,1547219408671,2019-01-11T16:10:08.671,0.24,-0.012,0.971,-0.084
4,1547219408751,2019-01-11T16:10:08.751,0.32,-0.013,0.954,-0.094
5,1547219408831,2019-01-11T16:10:08.831,0.40,-0.017,0.902,-0.108
6,1547219408911,2019-01-11T16:10:08.911,0.48,-0.025,0.855,-0.128
7,1547219408991,2019-01-11T16:10:08.991,0.56,-0.042,0.844,-0.139
8,1547219409071,2019-01-11T16:10:09.071,0.64,-0.049,0.875,-0.171
9,1547219409151,2019-01-11T16:10:09.151,0.72,-0.075,0.871,-0.139


In [148]:
files = glob("../../data/raw/MetaMotion/*.csv")

data_path = "../../data/raw/MetaMotion\\"
f = files[0]

In [149]:
participant = f.split("-")[0].replace(data_path,"")
label = f.split("-")[1]
category = f.split("-")[2].rstrip("123")

In [150]:
df = pd.read_csv(f)
df["participant"] = participant
df["label"] = label
df["category"] = category


In [151]:
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

acc_set = 1
gyr_set = 1

for f in files:
    participant = f.split("-")[0].replace(data_path,"")
    label = f.split("-")[1]
    category = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")
    
    df = pd.read_csv(f)
    df["participant"] = participant
    df["label"] = label
    df["category"] = category

    if "Accelerometer" in f:
        df["set"] = acc_set
        acc_set += 1
        acc_df = pd.concat([acc_df,df])
    if "Gyroscope" in f:
        df["set"] = gyr_set
        gyr_set += 1
        gyr_df = pd.concat([gyr_df,df])


In [152]:
acc_df.index = pd.to_datetime(acc_df['epoch (ms)'], unit="ms")
gyr_df.index =  pd.to_datetime(gyr_df['epoch (ms)'], unit="ms")

acc_df = acc_df.drop(['epoch (ms)',	'time (01:00)', 'elapsed (s)'], axis=1)
gyr_df = gyr_df.drop(['epoch (ms)',	'time (01:00)', 'elapsed (s)'], axis=1)

In [153]:
merged_data = pd.concat([acc_df.iloc[:,:3], gyr_df], axis=1)
merged_data.columns = ["acc_x",
                       "acc_y",
                       "acc_z",
                       "gyr_x",
                       "gyr_y",
                       "gyr_z",
                       "label",
                       "category",
                       "participant",
                       "set"]

## Frequency conversion

In [154]:
sampling = {'acc_x':'mean',
            'acc_y':'mean',
            'acc_z':'mean',
            'gyr_x':'mean',
            'gyr_y':'mean',
            'gyr_z':'mean',
            'label':'last',
            'category':'last',
            'participant':'last',
            'set':'last'}

# A Grouper allows the user to specify a groupby instruction for an object.
# freq : str / frequency object, defaults to None. This will groupby the specified frequency if the target selection (via key or level) is a datetime-like object.
days = [g for n,g in merged_data.groupby(pd.Grouper(freq="D"))]
data_sampled = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])



In [155]:
data_sampled["set"] = data_sampled["set"].astype("int")
data_sampled.info()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   label        9009 non-null   object 
 7   category     9009 non-null   object 
 8   participant  9009 non-null   object 
 9   set          9009 non-null   int32  
dtypes: float64(6), int32(1), object(3)
memory usage: 739.0+ KB


In [156]:
# Export dataset
data_sampled.to_pickle("../../data/intermediate/01_data_processed.pkl")